-1. Pre-steps to make sure it runs.

In [71]:
import pypsa 
import sys
sys.path.append("..")  # or "../src" if the notebook is inside /notebooks

0. Load in example network

In [72]:
n_rd = pypsa.Network()
z_da = pypsa.Network()
# Get the networks from files:
folder = "C:/Users/ameld/thesis_local/PowerVision/output_networks/FBMC"
n_rd.import_from_netcdf(f"{folder}/nodal.nc")
z_da.import_from_netcdf(f"{folder}/zonal.nc")

INFO:pypsa.io:Imported network nodal.nc has buses, generators, lines, loads
INFO:pypsa.io:Imported network zonal.nc has buses, carriers, generators, links, loads


1. Calculate parameters using fbmc_params.py

In [73]:
z_da.optimize(solver_name='gurobi')

c:\Users\ameld\thesis_local\pypsa-fbmc\venv\Lib\site-packages\linopy\common.py:154: UserWarning: coords for dimension(s) ['Generator'] is not aligned with the pandas object. Previously, the indexes of the pandas were ignored and overwritten in these cases. Now, the pandas object's coordinates are taken considered for alignment.
  warn(
INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.io: Writing time: 0.02s


Set parameter Username


INFO:gurobipy:Set parameter Username


Set parameter LicenseID to value 2594984


INFO:gurobipy:Set parameter LicenseID to value 2594984


...

Solved in 1 iterations and 0.01 seconds (0.00 work units)


INFO:gurobipy:Solved in 1 iterations and 0.01 seconds (0.00 work units)


Optimal objective  9.750000000e+05


INFO:gurobipy:Optimal objective  9.750000000e+05
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 9 primals, 21 duals
Objective: 9.75e+05
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Link-fix-p-lower, Link-fix-p-upper were not assigned to the network.


('ok', 'optimal')

## 1. NTC

In [74]:
# Add constraint representing NTC:

# Define NTC values
NTC_PERCENTAGE = 0.75 # At 0.75, flow is 1000 MW, same as FBMC w/o FRM.
# At (0.675), flow keeps under 900 MW, same as FBMC w/ FRM.
ntc_values = z_da.links["p_nom"] * NTC_PERCENTAGE
ntc_values

import xarray as xr
ntc_values_xr = xr.DataArray(
    ntc_values,
    dims=["Link-fix"],
    coords={"Link-fix": z_da.links.index.to_list()})

# Update the constraint
ntc_constraint = z_da.model.constraints["Link-fix-p-upper"].lhs <= ntc_values_xr

z_da.model.add_constraints(ntc_constraint, name="Link-fix-p-upper-NTC")
# z_da.model.remove_constraints("Link-fix-p-upper")

Constraint `Link-fix-p-upper-NTC` [snapshot: 1, Link-fix: 6]:
-------------------------------------------------------------
[now, NO1_NO2]: +1 Link-p[now, NO1_NO2] ≤ 750.0
[now, NO2_NO1]: +1 Link-p[now, NO2_NO1] ≤ 750.0
[now, NO1_NO3]: +1 Link-p[now, NO1_NO3] ≤ 750.0
[now, NO3_NO1]: +1 Link-p[now, NO3_NO1] ≤ 750.0
[now, NO2_NO3]: +1 Link-p[now, NO2_NO3] ≤ 750.0
[now, NO3_NO2]: +1 Link-p[now, NO3_NO2] ≤ 750.0

In [75]:
z_da.model.solve(solver_name='gurobi')

INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.io: Writing time: 0.03s


Set parameter Username


INFO:gurobipy:Set parameter Username


Set parameter LicenseID to value 2594984


INFO:gurobipy:Set parameter LicenseID to value 2594984


...

Solved in 1 iterations and 0.02 seconds (0.00 work units)


INFO:gurobipy:Solved in 1 iterations and 0.02 seconds (0.00 work units)


Optimal objective  1.325000000e+06


INFO:gurobipy:Optimal objective  1.325000000e+06
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 9 primals, 27 duals
Objective: 1.32e+06
Solver model: available
Solver message: 2



('ok', 'optimal')

In [76]:
z_da.model.solution.variables['Generator-p']

<xarray.Variable (snapshot: 1, Generator: 3)> Size: 24B
array([[1500.,  750.,  500.]])

In [77]:
z_da.model.constraints['Link-fix-p-upper-NTC']

Constraint `Link-fix-p-upper-NTC` [snapshot: 1, Link-fix: 6]:
-------------------------------------------------------------
[now, NO1_NO2]: +1 Link-p[now, NO1_NO2] ≤ 750.0
[now, NO2_NO1]: +1 Link-p[now, NO2_NO1] ≤ 750.0
[now, NO1_NO3]: +1 Link-p[now, NO1_NO3] ≤ 750.0
[now, NO3_NO1]: +1 Link-p[now, NO3_NO1] ≤ 750.0
[now, NO2_NO3]: +1 Link-p[now, NO2_NO3] ≤ 750.0
[now, NO3_NO2]: +1 Link-p[now, NO3_NO2] ≤ 750.0

In [78]:
z_da.model.objective

Objective:
----------
LinearExpression: +300 Generator-p[now, gen_1_hydro] + 500 Generator-p[now, gen_2_hydro] + 1000 Generator-p[now, gen_3_hydro]
Sense: min
Value: 1325000.0

In [79]:
z_da.model.remove_constraints("Link-fix-p-upper-NTC")

In [80]:
z_da.model.solution['Link-p'].to_dataframe()

Link-p
snapshot Link           
now      NO1_NO2   750.0
         NO2_NO1     0.0
         NO1_NO3   750.0
         NO3_NO1     0.0
         NO2_NO3   750.0
         NO3_NO2     0.0

## 2. FBMC


In [81]:
from src.fbmc.config import FBMCConfig
import src.fbmc as fbmc_main
import importlib

importlib.reload(fbmc_main)

config = FBMCConfig()
config.reliability_margin_factor = 0.0
config.gsk_method = "ADJUSTABLE_CAP"

z_da = fbmc_main.setup_fbmc_model(
    basecase_nodal_network= n_rd, 
    target_zonal_network=z_da,
    config=config
)

In [82]:
z_da.model.constraints['CNE-RAM']

Constraint `CNE-RAM` [CNE: 3, snapshot: 1]:
-------------------------------------------
[line_1_2, now]: +0 Zone-p[now, NO1] - 0.6667 Zone-p[now, NO2] - 0.3333 Zone-p[now, NO3] ≤ -166.66666666666697
[line_1_3, now]: +0 Zone-p[now, NO1] - 0.3333 Zone-p[now, NO2] - 0.6667 Zone-p[now, NO3] ≤ -583.3333333333331
[line_2_3, now]: +0 Zone-p[now, NO1] + 0.3333 Zone-p[now, NO2] - 0.3333 Zone-p[now, NO3] ≤ 583.333333333333

In [83]:
n_rd.carriers

attribute,co2_emissions,color,nice_name,max_growth,max_relative_growth
Carrier,,,,,


In [84]:
z_da.model.solve(solver_name='gurobi')

INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.io: Writing time: 0.02s


Set parameter Username


INFO:gurobipy:Set parameter Username


Set parameter LicenseID to value 2594984


INFO:gurobipy:Set parameter LicenseID to value 2594984


...

Solved in 1 iterations and 0.01 seconds (0.00 work units)


INFO:gurobipy:Solved in 1 iterations and 0.01 seconds (0.00 work units)


Optimal objective  1.175000000e+06


INFO:gurobipy:Optimal objective  1.175000000e+06
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 6 primals, 13 duals
Objective: 1.18e+06
Solver model: available
Solver message: 2



('ok', 'optimal')

In [85]:
z_da.model.constraints['CNE-RAM']

Constraint `CNE-RAM` [CNE: 3, snapshot: 1]:
-------------------------------------------
[line_1_2, now]: -0.6667 Zone-p[now, NO2] - 0.3333 Zone-p[now, NO3] ≤ -166.66666666666697
[line_1_3, now]: -0.3333 Zone-p[now, NO2] - 0.6667 Zone-p[now, NO3] ≤ -583.3333333333331
[line_2_3, now]: +0.3333 Zone-p[now, NO2] - 0.3333 Zone-p[now, NO3] ≤ 583.333333333333

In [86]:
z_da.model.constraints['Zonal_balance']

Constraint `Zonal_balance` [snapshot: 1]:
-----------------------------------------
[now]: +1 Zone-p[now, NO1] + 1 Zone-p[now, NO2] + 1 Zone-p[now, NO3] = 2750.0

In [87]:
z_da.model.constraints["CNE-RAM"]

Constraint `CNE-RAM` [CNE: 3, snapshot: 1]:
-------------------------------------------
[line_1_2, now]: -0.6667 Zone-p[now, NO2] - 0.3333 Zone-p[now, NO3] ≤ -166.66666666666697
[line_1_3, now]: -0.3333 Zone-p[now, NO2] - 0.6667 Zone-p[now, NO3] ≤ -583.3333333333331
[line_2_3, now]: +0.3333 Zone-p[now, NO2] - 0.3333 Zone-p[now, NO3] ≤ 583.333333333333

In [88]:
opt_gens = z_da.model.solution['Generator-p']
opt_gens_df = opt_gens.to_dataframe().unstack('Generator')
opt_gens_df.columns = opt_gens_df.columns.droplevel(0)
opt_gens_df

Generator,gen_1_hydro,gen_2_hydro,gen_3_hydro
snapshot,,,
now,1000.0,1750.0,0.0


In [89]:
z_da.generators_t.p

Generator,gen_1_hydro,gen_2_hydro,gen_3_hydro
snapshot,,,
now,2000.0,750.0,0.0


In [90]:
solution_n_rd = n_rd.copy(snapshots=n_rd.snapshots)
solution_n_rd.generators_t.p_set = opt_gens_df


FBMC Line values

In [91]:
solution_n_rd.pf()
solution_n_rd.lines_t.p0

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network <pypsa.components.SubNetwork object at 0x000002E7EC251350> for snapshots Index(['now'], dtype='object', name='snapshot')


,line_1_2,line_1_3,line_2_3
snapshot,,,
now,-0.000071,1000.000071,999.999929


NTC line values:

In [92]:
n_rd.lines_t.p0

Line,line_1_2,line_1_3,line_2_3
snapshot,,,
now,333.333333,666.666667,333.333333
